In [ ]:
!pip install langchain

In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
with open('/content/generative ai.txt') as f:
  text_gen_ai = f.read()

In [ ]:
# 읽어온 파일을 잘라보도록 해요~
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    # 문단 단위로 나누어야 함으로! 개행문자 2번이 문단일 것이다,
    separator='\n\n',

    # 자르는 용량
    chunk_size = 1000,

    # 연속되는 문장, 문단, 글임을 알려주기 위해 일정 비율을 겹치게 만들어서 잘라줌
    # 그 겹치는 비율을 정의해줌
    chunk_overlap = 100,

    # chunk size를 무엇을 기준으로 셀거야?를 지정해준다.
    length_function = len
)

In [ ]:
texts = text_splitter.split_text(text_gen_ai)

In [ ]:
print(texts[0])
print('-' * 100)
print(texts[1])
print('-' * 100)
print(texts[2])

What is generative AI?
Generative AI can learn from existing artifacts to generate new, realistic artifacts (at scale) that reflect the characteristics of the training data but don’t repeat it. It can produce a variety of novel content, such as images, video, music, speech, text, software code and product designs.  
Generative AI uses a number of techniques that continue to evolve. Foremost are AI foundation models, which are trained on a broad set of unlabeled data that can be used for different tasks, with additional fine-tuning. Complex math and enormous computing power are required to create these trained models, but they are, in essence, prediction algorithms. 
Today, generative AI most commonly creates content in response to natural language requests — it doesn’t require knowledge of or entering code — but the enterprise use cases are numerous and include innovations in drug and chip design and material science development. (Also see “What are some practical uses of generative AI

In [ ]:
char_list = []

# 문단 수 만큼 반복한다.
for i in range(len(texts)):
  char_list.append(len(texts[i]))

# 페이지를 자른 모습을 보여준다.
print(char_list)

[1003, 2238, 5022, 3460, 1265, 995, 1142, 2523, 3101]


In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf')
pages = loader.load_and_split()

txt 파일 같은 경우는 page의 개념이 없기에 문단 단위로 split을 했지만 pdf는 page가 있기 때문에 페이지로 split

In [ ]:
# page 단위로 잘랐을 때 총 페이지의 개수
len(pages)

19

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = len
)

In [ ]:
# txt file -> split txt
# pdf file -> split document
texts = text_splitter.split_documents(pages)

In [ ]:
# 500자 이내로 text를 잘라줍니다!
char_list = []
for i in range(len(texts)):
  char_list.append(len(texts[i].page_content))

print(char_list)

[175, 239, 458, 491, 304, 495, 448, 415, 452, 452, 325, 475, 444, 324, 500, 441, 321, 477, 497, 139, 465, 431, 151, 471, 412, 458, 453, 105, 476, 345, 499, 190, 479, 417, 465, 380, 478, 388, 487, 307, 366, 78]


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = len
)

In [ ]:
# 500자 이내로 적절하게 글을 쪼개준다.

texts = text_splitter.create_documents([text_gen_ai])
print("글자 수:", len(texts[0].page_content))
texts[0].page_content

글자 수: 315


'What is generative AI?\nGenerative AI can learn from existing artifacts to generate new, realistic artifacts (at scale) that reflect the characteristics of the training data but don’t repeat it. It can produce a variety of novel content, such as images, video, music, speech, text, software code and product designs.'

In [ ]:
char_list = []
for i in range(len(texts)):
  char_list.append(len(texts[i].page_content))


print("페이지 별 글자 수:", char_list)
print("총 페이지 수:", len(char_list))

페이지 별 글자 수: [315, 355, 328, 464, 194, 432, 239, 497, 181, 272, 285, 488, 386, 378, 372, 491, 421, 474, 452, 463, 471, 318, 345, 430, 391, 327, 471, 276, 411, 282, 251, 321, 495, 410, 356, 496, 497, 374, 499, 171, 141, 436, 413, 388, 489, 431, 359, 422, 444, 499, 96, 321, 198, 488, 420, 248]
총 페이지 수: 56


RecursiveCharacterTextSplitter를 주로 많이 쓴다.\
chunk_size 아래로 페이지를 쪼개준다.\
우리가 원하는 사이즈가 될 때까지 쪼개면서 오류가 나지 않도록 반복하여 쪼개기를 수행한다.

## 토큰 기반 Splitter

In [28]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [30]:
import tiktoken

In [31]:
# tokenizer 만들기
tokenizer = tiktoken.get_encoding('cl100k_base')

# return token 개수 -> encode 사
def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [32]:
print(texts[1].page_content) # split한 text 출력
print(len(texts[1].page_content)) # 문자의 수 출력
print(tiktoken_len(texts[1].page_content)) # token의 수 출력

Generative AI uses a number of techniques that continue to evolve. Foremost are AI foundation models, which are trained on a broad set of unlabeled data that can be used for different tasks, with additional fine-tuning. Complex math and enormous computing power are required to create these trained models, but they are, in essence, prediction algorithms.
355
70


In [38]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 50,
    length_function = tiktoken_len # 우리가 token의 개수를 세는 함수로 기준을 바꿔 줌!
)

In [39]:
texts = text_splitter.split_documents(pages)
print(len(texts[1].page_content))
print(tiktoken_len(texts[1].page_content))

75
94


한글 문서여서 토큰 개수가 단어의 개수보다 많아지는 문제! -> 한국어가 유니코드 문자이기 때문이다.\ 따라서 한국어 쓸 때는 주의가 필요하다.